This code was developing how to interact with the sensor and animate plotting. It can both generate random data to plot or plot from sensor data. The random data was used to develop plotting before being able to interact with the light sensor

In [ ]:
import datetime as dt

import serial
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.dates as mdates
from collections import deque

In [ ]:
%matplotlib widget
HISTORY_SIZE = 25

colors = {
    'temp': deque(maxlen=HISTORY_SIZE),
    'violet': deque(maxlen=HISTORY_SIZE),
    'blue': deque(maxlen=HISTORY_SIZE),
    'green': deque(maxlen=HISTORY_SIZE),
    'yellow': deque(maxlen=HISTORY_SIZE),
    'orange': deque(maxlen=HISTORY_SIZE),
    'red': deque(maxlen=HISTORY_SIZE),
}

def readSensor(serialPort, colors):
    serialData = ser.readline()
    if serialData == b'\r\n':
        serialData = ser.readline()
    data = serialData.split()
    if len(data) != 14:
        return

    colors['temp'].append(int(data[1]))
    colors['violet'].append(int(data[3]))
    colors['blue'].append(int(data[5]))
    colors['green'].append(int(data[7]))
    colors['yellow'].append(int(data[9]))
    colors['orange'].append(int(data[11]))
    colors['red'].append(int(data[13]))
        
ser = serial.Serial('/dev/ttyACM0', 115200)
xvals   = deque(maxlen=HISTORY_SIZE)


fig, ax = plt.subplots()
plt.tight_layout()

def animate(i):
    readSensor(ser)
    
    x_vals = [dt.datetime.now() + dt.timedelta(seconds=i) for i in range(len(reds))]
    
    ax.cla()
    fig.autofmt_xdate()
    
    ax.plot(x_vals, violets, color='darkviolet')
    ax.plot(x_vals, blues, color='b')
    ax.plot(x_vals, greens, color='g')
    ax.plot(x_vals, yellows, color='yellow')
    ax.plot(x_vals, oranges, color='orange')
    ax.plot(x_vals, reds, color='r')

ani = FuncAnimation(fig, animate, interval=100)

In [1]:
%matplotlib widget

import datetime as dt
import serial
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from collections import deque
from scipy.stats import norm

HISTORY_SIZE = 25
#ser = serial.Serial('/dev/ttyACM0', 115200)

readings = deque(maxlen=HISTORY_SIZE)
y_vals = deque(maxlen=HISTORY_SIZE)
x_vals = np.arange(350, 750, 1)

sd = 16.985
violet_pdf = lambda x: norm.pdf(x, 450, sd)
blue_pdf = lambda x: norm.pdf(x, 500, sd)
green_pdf = lambda x: norm.pdf(x, 550, sd)
yellow_pdf = lambda x: norm.pdf(x, 570, sd)
orange_pdf = lambda x: norm.pdf(x, 600, sd)
red_pdf = lambda x: norm.pdf(x, 650, sd)

def read_sensor(ser):
    data = ser.readline()
    if data == b'\r\n':
        data = ser.readline()
    data = data.split()
    if len(data != 14):
        return None
    
    colors = {}
    colors['violet'] = int(data[3])
    colors['blue'] = int(data[5])
    colors['green'] = int(data[7])
    colors['yellow'] = int(data[9])
    colors['orange'] = int(data[11])
    colors['red'] = int(data[13])
    
    return colors

def gen_random_data():
    colors = {}
    colors['violet'] = np.random.uniform(low=0, high=1500)
    colors['blue'] = np.random.uniform(low=0, high=1500)
    colors['green'] = np.random.uniform(low=0, high=1500)
    colors['yellow'] = np.random.uniform(low=0, high=1500)
    colors['orange'] = np.random.uniform(low=0, high=1500)
    colors['red'] = np.random.uniform(low=0, high=1500)
    
    return colors

def gen_chart_data(reading):
    chart_data = [
        reading['violet'] * violet_pdf(x) + \
        reading['blue'] * blue_pdf(x) + \
        reading['green'] * green_pdf(x) + \
        reading['orange'] * orange_pdf(x) + \
        reading['red'] * red_pdf(x)
        for x in x_vals
    ]
    
    return chart_data

fig, ax = plt.subplots()
plt.tight_layout()   

def animate(i):
    #reading = read_sensor(ser)
    reading = gen_random_data()
    if reading is not None:
        chart_data = convert_to_chart_data(reading)
        
    # y_vals = [dt.datetime.now() + dt.timedelta(seconds=i) for i in range(len(readings))]
    ax.cla()
    ax.plot(x_vals, chart_data)

ani = FuncAnimation(fig, animate, interval=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …